In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset

In [15]:
# Load training, validation, and test data
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_val = pd.read_csv('X_val.csv')
y_val = pd.read_csv('y_val.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [16]:
# Convert to numpy arrays
X_train_np = X_train.values
y_train_np = y_train.values.flatten()
X_val_np = X_val.values
y_val_np = y_val.values.flatten()
X_test_np = X_test.values
y_test_np = y_test.values.flatten()

In [17]:
# Feature selection using PCA to reduce features
n_components = 10 
pca = PCA(n_components=n_components)
X_train_reduced = pca.fit_transform(X_train_np)
X_val_reduced = pca.transform(X_val_np)
X_test_reduced = pca.transform(X_test_np)

In [18]:
# Reshape data to fit CNN input format (batch_size, channels, height, width)
X_train_reduced = X_train_reduced.reshape((X_train_reduced.shape[0], 1, 1, n_components))
X_val_reduced = X_val_reduced.reshape((X_val_reduced.shape[0], 1, 1, n_components))
X_test_reduced = X_test_reduced.reshape((X_test_reduced.shape[0], 1, 1, n_components))


In [19]:
# Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(1, 3))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(1, 3))
        self.fc1 = nn.Linear(32 * 1 * (n_components - 4), 128)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

In [20]:
# Initialize model, loss function, and optimizer
model = CNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [21]:
# Convert data to PyTorch tensors with float32 dtype
X_train_tensor = torch.tensor(X_train_reduced, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_reduced, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_reduced, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.float32)


In [22]:
# Create DataLoader for batch processing
batch_size = 1000 
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


In [23]:
# Lists to log training loss and accuracy
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

In [24]:
# Training loop with batch processing
epochs = 400
for epoch in range(epochs):
    model.train()
    running_loss = 0.0  # Reset running loss for each epoch
    
    # Process each batch
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(batch_x).squeeze()  # Remove singleton dimension
        loss = criterion(predictions, batch_y)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()# Accumulate batch loss

    # Calculate the average loss for the epoch
    epoch_loss = (running_loss / len(train_loader))
    
    # Calculate accuracy and loss for validation set
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # No need to track gradients during evaluation
        y_val_pred = model(X_val_tensor).squeeze()
        val_loss = criterion(y_val_pred, y_val_tensor).item()
        val_mse = mean_squared_error(y_val_np, y_val_pred.numpy())
        val_r2 = r2_score(y_val_np, y_val_pred.numpy())  # R² score
        val_accuracy = val_r2 * 100  # Convert to percentage

        # Store logs for this epoch
        train_losses.append(epoch_loss)
        val_losses.append(val_loss)
        train_accuracies.append(val_accuracy)
        val_accuracies.append(val_accuracy)

    # Print results for each epoch
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss}')

Epoch [1/400], Loss: 0.18143770337104798
Epoch [2/400], Loss: 0.0849282719194889
Epoch [3/400], Loss: 0.042863625884056095
Epoch [4/400], Loss: 0.03701202601194382
Epoch [5/400], Loss: 0.033344348073005674
Epoch [6/400], Loss: 0.028579869642853736
Epoch [7/400], Loss: 0.02354653097689152
Epoch [8/400], Loss: 0.018677205964922904
Epoch [9/400], Loss: 0.014252216964960098
Epoch [10/400], Loss: 0.010274343565106392
Epoch [11/400], Loss: 0.007034100219607353
Epoch [12/400], Loss: 0.0047370801120996476
Epoch [13/400], Loss: 0.00338000594638288
Epoch [14/400], Loss: 0.0026107383891940117
Epoch [15/400], Loss: 0.0021984754782170056
Epoch [16/400], Loss: 0.001948431204073131
Epoch [17/400], Loss: 0.0018111222283914686
Epoch [18/400], Loss: 0.0016979382606223225
Epoch [19/400], Loss: 0.0016123669454827904
Epoch [20/400], Loss: 0.0015365476114675403
Epoch [21/400], Loss: 0.001483995197340846
Epoch [22/400], Loss: 0.001440093843266368
Epoch [23/400], Loss: 0.001404731017537415
Epoch [24/400], Los

In [25]:
# Save trained model and logs
torch.save({
    'model_state_dict': model.state_dict(),
    'train_losses': train_losses,
    'train_accuracies': train_accuracies,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies
}, 'cnn_model.pth')

In [26]:
# Testing on the test set
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor).squeeze().numpy()
    
    # Calculate MSE and R² score
    test_mse = mean_squared_error(y_test_np, y_test_pred)
    test_r2 = r2_score(y_test_np, y_test_pred)  # R² score

    # Calculate accuracy as R² score in percentage
    test_accuracy = test_r2 * 100
    
    print(f'Mean Squared Error on test data: {test_mse}')
    print(f'R² score (Accuracy) on test data: {test_r2}')
    print(f'Accuracy: {test_accuracy}%')

Mean Squared Error on test data: 0.0008419479059511409
R² score (Accuracy) on test data: 0.9785310755562355
Accuracy: 97.85310755562355%
